# Controller Debug
This code allows us to figure out any oddities in construction of the controller, including maximums and centering.

In [ ]:
import evdev, time
joy = evdev.InputDevice('/dev/input/event14')
print(joy)

In [ ]:
print(f"Left Stick X:  {joy.absinfo(evdev.ecodes.ABS_X)}")
print(f"Left Stick Y:  {joy.absinfo(evdev.ecodes.ABS_Y)}")
print(f"Right Stick X: {joy.absinfo(evdev.ecodes.ABS_RX)}")
print(f"Right Stick Y: {joy.absinfo(evdev.ecodes.ABS_RY)}")

In [ ]:
now = time.time()
file = open('controller_debug_data.csv', 'wt', newline='\n')
while True:
    file.write(f"{time.time()-now},{joy.absinfo(evdev.ecodes.ABS_X).value},{joy.absinfo(evdev.ecodes.ABS_Y).value},{joy.absinfo(evdev.ecodes.ABS_RX).value},{joy.absinfo(evdev.ecodes.ABS_RY).value}\n")
    time.sleep(0.005)